## Tarea 2 - Aprendizaje Automático
Vicente Mieres

In [ ]:
# importacion de librerias

import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


# Parte 1 - Lectura de Imágenes

In [ ]:
def load_image_paths_and_labels(file_path):
    """Carga las rutas de las imágenes y las etiquetas desde un archivo de texto."""
    image_paths = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file.readlines():
            path, label = line.strip().split('\t')  
            image_paths.append(path)
            labels.append(int(label)) 
    return image_paths, np.array(labels)